# 初めてのインテリジェントエージェントチームを構築する：ADK を使用した段階的な天気予報ボット

シンプルな基盤の上に高度な機能を段階的に重ねながら、**天気予報ボットエージェントチーム**の構築に着手します。天気を調べることができる単一のエージェントから始めて、次のような機能を徐々に追加していきます。

*   さまざまな AI モデル（Gemini、GPT、Claude）の活用。
*   特定のタスク（挨拶や別れなど）のための専用サブエージェントの設計。
*   エージェント間のインテリジェントな委任の有効化。
*   永続的なセッション状態を使用したエージェントへのメモリの付与。
*   コールバックを使用した重要な安全ガードレールの実装。

**なぜ天気予報ボットチームなのか？**

このユースケースは、一見シンプルに見えますが、複雑な実世界の代理アプリケーションを構築するために不可欠な ADK のコアコンセプトを探求するための実用的で関連性のあるキャンバスを提供します。インタラクションの構造化、状態の管理、安全性の確保、そして連携して動作する複数の AI「頭脳」の調整方法を学びます。

**ADK とは？**

念のため、ADK は、大規模言語モデル（LLM）を搭載したアプリケーションの開発を合理化するために設計された Python フレームワークです。推論、計画、ツールの利用、ユーザーとの動的な対話、チーム内での効果的な連携が可能なエージェントを作成するための堅牢なビルディングブロックを提供します。

**この高度なチュートリアルでは、以下を習得します。**

*   ✅ **ツールの定義と使用法:** エージェントに特定の能力（データの取得など）を付与する Python 関数（`tools`）を作成し、エージェントにそれらを効果的に使用する方法を指示します。
*   ✅ **マルチ LLM の柔軟性:** LiteLLM 統合を介して、さまざまな主要 LLM（Gemini、GPT-4o、Claude Sonnet）を利用するようにエージェントを構成し、各タスクに最適なモデルを選択できるようにします。
*   ✅ **エージェントの委任とコラボレーション:** 専用のサブエージェントを設計し、ユーザーのリクエストをチーム内の最も適切なエージェントに自動的にルーティング（`auto flow`）できるようにします。
*   ✅ **メモリのためのセッション状態:** `Session State` と `ToolContext` を利用して、エージェントが会話のターンをまたいで情報を記憶できるようにし、より文脈に沿ったインタラクションを実現します。
*   ✅ **コールバックによる安全ガードレール:** `before_model_callback` と `before_tool_callback` を実装して、事前定義されたルールに基づいてリクエスト/ツールの使用を検査、変更、またはブロックし、アプリケーションの安全性と制御を強化します。

**最終状態の期待:**

このチュートリアルを完了すると、機能的なマルチエージェント天気予報ボットシステムを構築できます。このシステムは、天気情報を提供するだけでなく、会話の丁寧なやり取りを処理し、最後に確認した都市を記憶し、ADK を使用して調整された定義済みの安全境界内で動作します。

**前提条件:**

*   ✅ **Python プログラミングの確かな理解。**
*   ✅ **大規模言語モデル（LLM）、API、およびエージェントの概念に関する知識。**
*   ❗ **重要：ADK クイックスタートチュートリアル（複数可）の完了、または ADK の基本（エージェント、ランナー、セッションサービス、基本的なツールの使用法）に関する同等の基礎知識。** このチュートリアルは、これらの概念を直接基礎としています。
*   ✅ 使用する予定の LLM の **API キー**（例：Gemini 用の Google AI Studio、OpenAI Platform、Anthropic Console）。


---

**実行環境に関する注意:**

このチュートリアルは、Google Colab、Colab Enterprise、Jupyter ノートブックなどのインタラクティブなノートブック環境向けに構成されています。次の点に留意してください。

*   **非同期コードの実行:** ノートブック環境は、非同期コードを異なる方法で処理します。`await`（イベントループがすでに実行されている場合に適しており、ノートブックで一般的です）または `asyncio.run()`（スタンドアロンの `.py` スクリプトとして実行する場合や特定のノートブック設定で必要な場合が多い）を使用した例が表示されます。コードブロックは、両方のシナリオのガイダンスを提供します。
*   **手動ランナー/セッション設定:** 手順には、`Runner` および `SessionService` インスタンスを明示的に作成することが含まれます。このアプローチは、エージェントの実行ライフサイクル、セッション管理、および状態の永続化をきめ細かく制御できるため、示されています。

**代替案：ADK の組み込みツール（Web UI / CLI / API サーバー）の使用**

ADK の標準ツールを使用してランナーとセッション管理を自動的に処理するセットアップをご希望の場合は、その目的に合わせて構成された同等のコードを[こちら](https://github.com/google/adk-docs/tree/main/examples/python/tutorial/agent_team/adk-tutorial)で見つけることができます。そのバージョンは、`adk web`（Web UI の場合）、`adk run`（CLI インタラクションの場合）、または `adk api_server`（API を公開する場合）などのコマンドで直接実行するように設計されています。その代替リソースで提供されている `README.md` の指示に従ってください。

---

**エージェントチームを構築する準備はできましたか？始めましょう！**

---

## ステップ 0: 事前準備

In [ ]:
# @title 必要な依存ライブラリをインストール
!pip install google-adk -q
!pip install litellm -q

print("Installation complete.")

import os
import asyncio
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For multi-model support
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types # For creating message Content/Parts

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.ERROR)

print("Libraries imported.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/

In [ ]:
# @title APIキーの設定

# --- IMPORTANT: Replace placeholders with your real API keys ---

# Gemini API Key (Get from Google AI Studio: https://aistudio.google.com/app/apikey)
os.environ["GOOGLE_API_KEY"] = "YOUR_GOOGLE_API_KEY" # <--- REPLACE

# [Optional]
# OpenAI API Key (Get from OpenAI Platform: https://platform.openai.com/api-keys)
os.environ['OPENAI_API_KEY'] = 'YOUR_OPENAI_API_KEY' # <--- REPLACE

# [Optional]
# Anthropic API Key (Get from Anthropic Console: https://console.anthropic.com/settings/keys)
os.environ['ANTHROPIC_API_KEY'] = 'YOUR_ANTHROPIC_API_KEY' # <--- REPLACE

# --- Verify Keys (Optional Check) ---
print("API Keys Set:")
print(f"Google API Key set: {'Yes' if os.environ.get('GOOGLE_API_KEY') and os.environ['GOOGLE_API_KEY'] != 'YOUR_GOOGLE_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")
print(f"OpenAI API Key set: {'Yes' if os.environ.get('OPENAI_API_KEY') and os.environ['OPENAI_API_KEY'] != 'YOUR_OPENAI_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")
print(f"Anthropic API Key set: {'Yes' if os.environ.get('ANTHROPIC_API_KEY') and os.environ['ANTHROPIC_API_KEY'] != 'YOUR_ANTHROPIC_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")

# Configure ADK to use API keys directly (not Vertex AI for this multi-model setup)
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"


# @markdown **Security Note:** It's best practice to manage API keys securely (e.g., using Colab Secrets or environment variables) rather than hardcoding them directly in the notebook. Replace the placeholder strings above.

In [ ]:
# --- Define Model Constants for easier use ---

MODEL_GEMINI_2_0_FLASH = "gemini-2.0-flash"

# Note: Specific model names might change. Refer to LiteLLM/Provider documentation.
MODEL_GPT_4O = "openai/gpt-4o"
MODEL_CLAUDE_SONNET = "anthropic/claude-3-sonnet-20240229"


print("\nEnvironment configured.")

---

## ステップ 1: 初めてのエージェント - 基本的な天気情報検索

まず、天気予報ボットの基本的なコンポーネントである、特定のタスク（天気情報の検索）を実行できる単一のエージェントを構築することから始めましょう。これには、2 つのコア要素の作成が含まれます。

1. **ツール:** エージェントに天気データを取得する*能力*を与える Python 関数。
2. **エージェント:** ユーザーのリクエストを理解し、天気ツールを持っていることを認識し、いつ、どのように使用するかを決定する AI の「頭脳」。

---

###  1\. ツールの定義 (get_weather)

ADK では、ツールは、単なるテキスト生成を超えた具体的な機能をエージェントに与える構成要素です。これらは通常、API の呼び出し、データベースへのクエリ、計算の実行など、特定のアクションを実行する通常の Python 関数です。

最初のツールは、モックの天気予報を提供します。これにより、まだ外部 API キーを必要とせずに、エージェントの構造に集中できます。後で、このモック関数を実際の気象サービスを呼び出す関数に簡単に置き換えることができます。

重要なコンセプト：docstring は重要です！ エージェントの LLM は、関数の docstring に大きく依存して以下を理解します。

ツールが何をするか。
いつ使用するか。
どの引数が必要か（city: str）。
どの情報を返すか。
ベストプラクティス: ツールには、明確で、説明的で、正確な docstring を記述してください。これは、LLM がツールを正しく使用するために不可欠です。

In [ ]:
def get_weather(city: str) -> dict:
    """Retrieves the current weather report for a specified city.

    Args:
        city (str): The name of the city (e.g., "New York", "London", "Tokyo").

    Returns:
        dict: A dictionary containing the weather information.
              Includes a 'status' key ('success' or 'error').
              If 'success', includes a 'report' key with weather details.
              If 'error', includes an 'error_message' key.
    """
    print(f"--- Tool: get_weather called for city: {city} ---") # Log tool execution
    city_normalized = city.lower().replace(" ", "") # Basic normalization

    # Mock weather data
    mock_weather_db = {
        "newyork": {"status": "success", "report": "The weather in New York is sunny with a temperature of 25°C."},
        "london": {"status": "success", "report": "It's cloudy in London with a temperature of 15°C."},
        "tokyo": {"status": "success", "report": "Tokyo is experiencing light rain and a temperature of 18°C."},
    }

    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return {"status": "error", "error_message": f"Sorry, I don't have weather information for '{city}'."}

# Example tool usage (optional test)
print(get_weather("New York"))
print(get_weather("Paris"))

### 2\. エージェントの定義 (`weather_agent`)

次に、**エージェント**自体を作成しましょう。ADK の `Agent` は、ユーザー、LLM、および利用可能なツール間のインタラクションを調整します。

いくつかの主要なパラメータで構成します。

* `name`: このエージェントの一意の識別子（例: "weather\_agent\_v1"）。
* `model`: 使用する LLM を指定します（例: `MODEL_GEMINI_2_0_FLASH`）。特定の Gemini モデルから始めます。
* `description`: エージェントの全体的な目的の簡潔な要約。これは、他のエージェントが *この* エージェントにタスクを委任するかどうかを決定する必要がある場合に後で重要になります。
* `instruction`: LLM の動作方法、ペルソナ、目標、特に割り当てられた `tools` を *いつ、どのように* 利用するかについての詳細なガイダンス。
* `tools`: エージェントが使用を許可されている実際の Python ツール関数を含むリスト（例: `[get_weather]`）。

**ベストプラクティス:** 明確で具体的な `instruction` プロンプトを提供してください。指示が詳細であるほど、LLM はその役割とツールの効果的な使用方法をよりよく理解できます。必要に応じてエラー処理について明示的に記述してください。

**ベストプラクティス:** 説明的な `name` と `description` の値を選択してください。これらは ADK によって内部的に使用され、自動委任（後で説明）などの機能に不可欠です。

In [ ]:
# Use one of the model constants defined earlier
AGENT_MODEL = MODEL_GEMINI_2_0_FLASH # Starting with Gemini

weather_agent = Agent(
    name="weather_agent_v1",
    model=AGENT_MODEL, # Can be a string for Gemini or a LiteLlm object
    description="Provides weather information for specific cities.",
    instruction="You are a helpful weather assistant. "
                "When the user asks for the weather in a specific city, "
                "use the 'get_weather' tool to find the information. "
                "If the tool returns an error, inform the user politely. "
                "If the tool is successful, present the weather report clearly.",
    tools=[get_weather], # Pass the function directly
)

print(f"Agent '{weather_agent.name}' created using model '{AGENT_MODEL}'.")

###  3\.ランナーとセッションサービスの設定

会話を管理し、エージェントを実行するには、さらに 2 つのコンポーネントが必要です。

* `SessionService`: さまざまなユーザーとセッションの会話履歴と状態を管理する責任があります。`InMemorySessionService` は、すべてをメモリに保存する単純な実装であり、テストや単純なアプリケーションに適しています。交換されたメッセージを追跡します。ステップ 4 で状態の永続化について詳しく説明します。
* `Runner`: ユーザー入力を処理し、エージェントを呼び出し、全体的なフローを管理するエンジンです。`SessionService` を使用して会話履歴を永続化および取得します。

In [ ]:
# --- Session Management ---
# Key Concept: SessionService stores conversation history & state.
# InMemorySessionService is simple, non-persistent storage for this tutorial.
session_service = InMemorySessionService()

# Define constants for identifying the interaction context
APP_NAME = "weather_tutorial_app"
USER_ID = "user_1"
SESSION_ID = "session_001" # Using a fixed ID for simplicity

# Create the specific session where the conversation will happen
session = session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

# --- Runner ---
# Key Concept: Runner orchestrates the agent execution loop.
runner = Runner(
    agent=weather_agent, # The agent we want to run
    app_name=APP_NAME,   # Associates runs with our app
    session_service=session_service # Uses our session manager
)
print(f"Runner created for agent '{runner.agent.name}'.")

### 4\. エージェントとの対話（メッセージの送信)

エージェント、ツール、ランナー、セッションサービスが設定されたので、最初のメッセージを送信してエージェントと対話する準備が整いました。

`runner.send_message()` 関数は、エージェントとの通信の中心です。主な引数は次のとおりです。

* `agent`: リクエストを処理するエージェント（この場合は `weather_agent`）。
* `message`: ユーザーが送信する実際のテキスト。
* `session_id` (オプション): 会話を一意に識別する文字列。指定しない場合、ランナーは通常、新しいセッションを作成するか、デフォルトのセッションを使用します。セッション ID を明示的に提供すると、特定の会話を追跡し、後で再開できます。
* `tools_context` (オプション): ツールの実行中に情報を渡したり、エージェントの呼び出し間で状態を維持したりするために使用できるディクショナリ。これについては、ステップ 4 で詳しく説明します。

**非同期実行に関する注意:**

ADK は非同期操作に大きく依存しており、特に LLM や外部ツールとの対話時に効率的です。Jupyter ノートブック（または既に実行中のイベントループを持つその他の環境）で作業している場合、通常、`await runner.send_message(...)` を使用して非同期呼び出しを直接待機できます。

ただし、標準の Python スクリプトから実行している場合、またはノートブックのイベントループが期待どおりに動作していない場合は、`asyncio.run(runner.send_message(...))` を使用して非同期関数を実行する必要があります。

提供されているコードスニペットには、両方のシナリオのコメントが含まれています。状況に合わせて適切な行のコメントを解除してください。このチュートリアルの残りの部分では、簡潔にするために `await` バージョンを主に表示しますが、原則は同じです。

In [ ]:
from google.genai import types # For creating message Content/Parts

async def call_agent_async(query: str, runner, user_id, session_id):
  """Sends a query to the agent and prints the final response."""
  print(f"\n>>> User Query: {query}")

  # Prepare the user's message in ADK format
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "Agent did not produce a final response." # Default

  # Key Concept: run_async executes the agent logic and yields Events.
  # We iterate through events to find the final answer.
  async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
      # You can uncomment the line below to see *all* events during execution
      # print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

      # Key Concept: is_final_response() marks the concluding message for the turn.
      if event.is_final_response():
          if event.content and event.content.parts:
             # Assuming text response in the first part
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # Handle potential errors/escalations
             final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
          # Add more checks here if needed (e.g., specific error codes)
          break # Stop processing events once the final response is found

  print(f"<<< Agent Response: {final_response_text}")

### 5\. テストしてみましょう！**

いくつかのメッセージを送信して、`weather_agent` がどのように応答するかを確認します。

*   「ニューヨークの天気は？」(ツールが成功するはずです)
*   「パリの天気は？」(ツールが存在しない都市のため、エラーを返すはずです)
*   「こんにちは」(ツールを使用するようにはトリガーされないはずです)

各応答で何が起こるかに注目してください。

*   LLM がユーザーの意図をどのように解釈するか。
*   `get_weather` ツールが呼び出されたかどうか (サーバーログ/出力で `--- Tool: get_weather called...` を探してください)。
*   エージェントがツールからの情報 (またはエラーメッセージ) をどのように最終的な応答に組み込むか。

In [ ]:
# We need an async function to await our interaction helper
async def run_conversation():
    await call_agent_async("What is the weather like in London?",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)

    await call_agent_async("How about Paris?",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID) # Expecting the tool's error message

    await call_agent_async("Tell me the weather in New York",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)

# Execute the conversation using await in an async context (like Colab/Jupyter)
await run_conversation()

# --- OR ---

# Uncomment the following lines if running as a standard Python script (.py file):
# import asyncio
# if __name__ == "__main__":
#     try:
#         asyncio.run(run_conversation())
#     except Exception as e:
#         print(f"An error occurred: {e}")

---

おめでとうございます！最初の ADK エージェントの構築と対話に成功しました。エージェントはユーザーのリクエストを理解し、ツールを使用して情報を見つけ、ツールの結果に基づいて適切に応答します。
次のステップでは、このエージェントを強化する基盤となる言語モデルを簡単に切り替える方法を探ります。


## [オプション]：LiteLLMによるマルチモデル化 [オプション]
ステップ1では、特定のGeminiモデルを利用した機能的な天気エージェントを構築しました。これは効果的ですが、実際のアプリケーションでは、_異なる_大規模言語モデル（LLM）を使用できる柔軟性がしばしば役立ちます。なぜでしょうか？
- **パフォーマンス:** 一部のモデルは特定のタスク（例：コーディング、推論、創造的な執筆）に優れています。
- **コスト:** モデルによって価格帯が異なります。
- **機能:** モデルは多様な機能、コンテキストウィンドウサイズ、ファインチューニングオプションを提供します。
- **可用性/冗長性:** 代替手段を持つことで、1つのプロバイダーで問題が発生した場合でもアプリケーションが機能し続けることが保証されます。

ADKは、[**LiteLLM**](https://github.com/BerriAI/litellm)ライブラリとの統合により、モデル間の切り替えをシームレスにします。LiteLLMは、100以上の異なるLLMへの一貫したインターフェースとして機能します。
**このステップでは、次のことを行います。**
1. `LiteLlm`ラッパーを使用して、OpenAI（GPT）やAnthropic（Claude）などのプロバイダーのモデルを使用するようにADK `Agent`を構成する方法を学びます。
2. それぞれ異なるLLMを搭載した天気エージェントのインスタンスを定義、構成し（独自のセッションとランナーを使用）、すぐにテストします。
3. これらの異なるエージェントと対話し、同じ基盤となるツールを使用していても、応答に潜在的なバリエーションがあることを観察します。


### 1\.LiteLlm` のインポート**

これは初期設定 (ステップ 0) でインポートしましたが、マルチモデルサポートの主要コンポーネントです。


In [ ]:
from google.adk.models.lite_llm import LiteLlm

### 2\. マルチモデルエージェントの定義**

モデル名の文字列だけを渡す（デフォルトでは Google の Gemini モデルになります）代わりに、目的のモデル識別子文字列を `LiteLlm` クラスでラップします。
- **キーコンセプト: `LiteLlm` ラッパー:** `LiteLlm(model="provider/model_name")` という構文は、このエージェントへのリクエストを LiteLLM ライブラリ経由で指定されたモデルプロバイダーにルーティングするように ADK に指示します。

ステップ0で OpenAI と Anthropic に必要な API キーを設定したことを確認してください。以前に定義した 関数（現在は `runner`、`user_id`、`session_id` を受け付けます）を使用して、各エージェントのセットアップ直後に対話します。 `call_agent_async`
以下の各ブロックは次の処理を行います。
- 特定の LiteLLM モデル（ または ）を使用してエージェントを定義します。 `MODEL_GPT_4O``MODEL_CLAUDE_SONNET`
- そのエージェントのテスト実行専用に、_新しく独立した_ `InMemorySessionService` とセッションを作成します。これにより、このデモンストレーションでは会話履歴が分離されます。
- 特定のエージェントとそのセッションサービス用に構成された `Runner` を作成します。
- すぐに を呼び出してクエリを送信し、エージェントをテストします。 `call_agent_async`

**ベストプラクティス:** タイプミスを避け、コードの管理を容易にするために、モデル名には（ステップ0で定義された 、 のような）定数を使用します。 `MODEL_GPT_4O``MODEL_CLAUDE_SONNET`
**エラー処理:** エージェント定義を `try...except` ブロックでラップします。これにより、特定のプロバイダーの API キーが見つからないか無効な場合に、コードセル全体が失敗するのを防ぎ、_構成されている_モデルでチュートリアルを続行できます。
まず、OpenAI の GPT-4o を使用してエージェントを作成し、テストしましょう。


In [ ]:
# Make sure 'get_weather' function from Step 1 is defined in your environment.
# Make sure 'call_agent_async' is defined from earlier.

# --- Agent using GPT-4o ---
weather_agent_gpt = None # Initialize to None
runner_gpt = None      # Initialize runner to None

try:
    weather_agent_gpt = Agent(
        name="weather_agent_gpt",
        # Key change: Wrap the LiteLLM model identifier
        model=LiteLlm(model=MODEL_GPT_4O),
        description="Provides weather information (using GPT-4o).",
        instruction="You are a helpful weather assistant powered by GPT-4o. "
                    "Use the 'get_weather' tool for city weather requests. "
                    "Clearly present successful reports or polite error messages based on the tool's output status.",
        tools=[get_weather], # Re-use the same tool
    )
    print(f"Agent '{weather_agent_gpt.name}' created using model '{MODEL_GPT_4O}'.")

    # InMemorySessionService is simple, non-persistent storage for this tutorial.
    session_service_gpt = InMemorySessionService() # Create a dedicated service

    # Define constants for identifying the interaction context
    APP_NAME_GPT = "weather_tutorial_app_gpt" # Unique app name for this test
    USER_ID_GPT = "user_1_gpt"
    SESSION_ID_GPT = "session_001_gpt" # Using a fixed ID for simplicity

    # Create the specific session where the conversation will happen
    session_gpt = session_service_gpt.create_session(
        app_name=APP_NAME_GPT,
        user_id=USER_ID_GPT,
        session_id=SESSION_ID_GPT
    )
    print(f"Session created: App='{APP_NAME_GPT}', User='{USER_ID_GPT}', Session='{SESSION_ID_GPT}'")

    # Create a runner specific to this agent and its session service
    runner_gpt = Runner(
        agent=weather_agent_gpt,
        app_name=APP_NAME_GPT,       # Use the specific app name
        session_service=session_service_gpt # Use the specific session service
        )
    print(f"Runner created for agent '{runner_gpt.agent.name}'.")

    # --- Test the GPT Agent ---
    print("\n--- Testing GPT Agent ---")
    # Ensure call_agent_async uses the correct runner, user_id, session_id
    await call_agent_async(query = "What's the weather in Tokyo?",
                           runner=runner_gpt,
                           user_id=USER_ID_GPT,
                           session_id=SESSION_ID_GPT)
    # --- OR ---

    # Uncomment the following lines if running as a standard Python script (.py file):
    # import asyncio
    # if __name__ == "__main__":
    #     try:
    #         asyncio.run(call_agent_async(query = "What's the weather in Tokyo?",
    #                      runner=runner_gpt,
    #                       user_id=USER_ID_GPT,
    #                       session_id=SESSION_ID_GPT)
    #     except Exception as e:
    #         print(f"An error occurred: {e}")

except Exception as e:
    print(f"❌ Could not create or run GPT agent '{MODEL_GPT_4O}'. Check API Key and model name. Error: {e}")


### 3\. テストしてみましょう！**

次に、Anthropic の Claude Sonnet についても同様に行います。

In [ ]:
# Make sure 'get_weather' function from Step 1 is defined in your environment.
# Make sure 'call_agent_async' is defined from earlier.

# --- Agent using Claude Sonnet ---
weather_agent_claude = None # Initialize to None
runner_claude = None      # Initialize runner to None

try:
    weather_agent_claude = Agent(
        name="weather_agent_claude",
        # Key change: Wrap the LiteLLM model identifier
        model=LiteLlm(model=MODEL_CLAUDE_SONNET),
        description="Provides weather information (using Claude Sonnet).",
        instruction="You are a helpful weather assistant powered by Claude Sonnet. "
                    "Use the 'get_weather' tool for city weather requests. "
                    "Analyze the tool's dictionary output ('status', 'report'/'error_message'). "
                    "Clearly present successful reports or polite error messages.",
        tools=[get_weather], # Re-use the same tool
    )
    print(f"Agent '{weather_agent_claude.name}' created using model '{MODEL_CLAUDE_SONNET}'.")

    # InMemorySessionService is simple, non-persistent storage for this tutorial.
    session_service_claude = InMemorySessionService() # Create a dedicated service

    # Define constants for identifying the interaction context
    APP_NAME_CLAUDE = "weather_tutorial_app_claude" # Unique app name
    USER_ID_CLAUDE = "user_1_claude"
    SESSION_ID_CLAUDE = "session_001_claude" # Using a fixed ID for simplicity

    # Create the specific session where the conversation will happen
    session_claude = session_service_claude.create_session(
        app_name=APP_NAME_CLAUDE,
        user_id=USER_ID_CLAUDE,
        session_id=SESSION_ID_CLAUDE
    )
    print(f"Session created: App='{APP_NAME_CLAUDE}', User='{USER_ID_CLAUDE}', Session='{SESSION_ID_CLAUDE}'")

    # Create a runner specific to this agent and its session service
    runner_claude = Runner(
        agent=weather_agent_claude,
        app_name=APP_NAME_CLAUDE,       # Use the specific app name
        session_service=session_service_claude # Use the specific session service
        )
    print(f"Runner created for agent '{runner_claude.agent.name}'.")

    # --- Test the Claude Agent ---
    print("\n--- Testing Claude Agent ---")
    # Ensure call_agent_async uses the correct runner, user_id, session_id
    await call_agent_async(query = "Weather in London please.",
                           runner=runner_claude,
                           user_id=USER_ID_CLAUDE,
                           session_id=SESSION_ID_CLAUDE)

    # --- OR ---

    # Uncomment the following lines if running as a standard Python script (.py file):
    # import asyncio
    # if __name__ == "__main__":
    #     try:
    #         asyncio.run(call_agent_async(query = "Weather in London please.",
    #                      runner=runner_claude,
    #                       user_id=USER_ID_CLAUDE,
    #                       session_id=SESSION_ID_CLAUDE)
    #     except Exception as e:
    #         print(f"An error occurred: {e}")


except Exception as e:
    print(f"❌ Could not create or run Claude agent '{MODEL_CLAUDE_SONNET}'. Check API Key and model name. Error: {e}")

両方のコードブロックからの出力を注意深く観察してください。次のことがわかるはずです。
1. 各エージェント（`weather_agent_gpt`、`weather_agent_claude`）は（APIキーが有効な場合）正常に作成されます。
2. それぞれに専用のセッションとランナーがセットアップされます。
3. 各エージェントは、クエリを処理する際に ツールを使用する必要性を正しく識別します（`--- Tool: get_weather called... ---` ログが表示されます）。 `get_weather`
4. _基礎となるツールロジック_は同一のままで、常にモックデータを返します。
5. ただし、各エージェントによって生成される**最終的なテキスト応答**は、言い回し、トーン、またはフォーマットがわずかに異なる場合があります。これは、指示プロンプトが異なるLLM（GPT-4o対Claude Sonnet）によって解釈および実行されるためです。

このステップは、ADKとLiteLLMが提供するパワーと柔軟性を示しています。コアアプリケーションロジック（ツール、基本的なエージェント構造）を一貫して維持しながら、さまざまなLLMを使用してエージェントを簡単に実験および展開できます。
次のステップでは、単一のエージェントを超えて、エージェントが互いにタスクを委任できる小さなチームを構築します。


---

## ステップ2：エージェントチームの構築 - 挨拶と別れの委任
ステップ1では、天気予報の検索のみに焦点を当てた単一のエージェントを構築し、実験しました。特定のタスクには効果的ですが、実際のアプリケーションでは、より多様なユーザーインタラクションを処理することがよくあります。単一の天気エージェントにツールや複雑な指示を追加し続けることもできますが、これはすぐに管理が難しくなり、効率も低下します。
より堅牢なアプローチは、**エージェントチーム**を構築することです。これには次のものが含まれます。
1. 特定の機能（例：天気用、挨拶用、計算用など）に合わせて設計された、複数の**特化型エージェント**を作成します。
2. 最初のユーザーリクエストを受信する**ルートエージェント**（またはオーケストレーター）を指定します。
3. ユーザーの意図に基づいて、ルートエージェントがリクエストを最も適切な特化型サブエージェントに**委任**できるようにします。

**なぜエージェントチームを構築するのか？**
- **モジュール性:** 個々のエージェントの開発、テスト、保守が容易になります。
- **専門化:** 各エージェントを特定のタスクに合わせて（指示、モデルの選択など）微調整できます。
- **スケーラビリティ:** 新しいエージェントを追加することで、新しい機能の追加が簡単になります。
- **効率性:** （挨拶のような）より単純なタスクには、より単純で安価なモデルを使用できる可能性があります。

**このステップでは、次のことを行います。**
1. 挨拶（）と別れ（）を処理するための単純なツールを定義します。 `say_hello``say_goodbye`
2. 2つの新しい特化型サブエージェントを作成します：`greeting_agent`と`farewell_agent`。
3. メインの天気エージェント（`weather_agent_v2`）を更新して、**ルートエージェント**として機能するようにします。
4. ルートエージェントをサブエージェントで構成し、**自動委任**を有効にします。
5. ルートエージェントにさまざまな種類のリクエストを送信して、委任フローをテストします。


### 1\. サブエージェント用のツールを定義する

まず、新しいスペシャリストエージェントのツールとして機能する単純なPython関数を作成しましょう。明確なdocstringは、それらを使用するエージェントにとって不可欠であることを忘れないでください


In [ ]:
# Ensure 'get_weather' from Step 1 is available if running this step independently.
# def get_weather(city: str) -> dict: ... (from Step 1)

def say_hello(name: str = "there") -> str:
    """Provides a simple greeting, optionally addressing the user by name.

    Args:
        name (str, optional): The name of the person to greet. Defaults to "there".

    Returns:
        str: A friendly greeting message.
    """
    print(f"--- Tool: say_hello called with name: {name} ---")
    return f"Hello, {name}!"

def say_goodbye() -> str:
    """Provides a simple farewell message to conclude the conversation."""
    print(f"--- Tool: say_goodbye called ---")
    return "Goodbye! Have a great day."

print("Greeting and Farewell tools defined.")

# Optional self-test
print(say_hello("Alice"))
print(say_goodbye())

### 2\.  サブエージェント（挨拶と別れ）を定義する**

次に、スペシャリスト向けの `Agent` インスタンスを作成します。非常に焦点が絞られた `instruction` と、特に重要なこととして、明確な `description` に注目してください。`description` は、_ルートエージェント_がこれらのサブエージェントに_いつ_委任するかを決定するために使用する主要な情報です。

**ベストプラクティス：** サブエージェントの `description` フィールドは、その特定の機能を正確かつ簡潔に要約する必要があります。これは、効果的な自動委任にとって非常に重要です。
**ベストプラクティス：** サブエージェントの `instruction` フィールドは、その限られた範囲に合わせて調整し、何をすべきか、何をすべきでないか（例：「あなたの_唯一の_タスクは...」）を正確に指示する必要があります。


In [ ]:
# If you want to use models other than Gemini, Ensure LiteLlm is imported and API keys are set (from Step 0/2)
# from google.adk.models.lite_llm import LiteLlm
# MODEL_GPT_4O, MODEL_CLAUDE_SONNET etc. should be defined
# Or else, continue to use: model = MODEL_GEMINI_2_0_FLASH

# --- Greeting Agent ---
greeting_agent = None
try:
    greeting_agent = Agent(
        # Using a potentially different/cheaper model for a simple task
        model = MODEL_GEMINI_2_0_FLASH,
        # model=LiteLlm(model=MODEL_GPT_4O), # If you would like to experiment with other models
        name="greeting_agent",
        instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting to the user. "
                    "Use the 'say_hello' tool to generate the greeting. "
                    "If the user provides their name, make sure to pass it to the tool. "
                    "Do not engage in any other conversation or tasks.",
        description="Handles simple greetings and hellos using the 'say_hello' tool.", # Crucial for delegation
        tools=[say_hello],
    )
    print(f"✅ Agent '{greeting_agent.name}' created using model '{greeting_agent.model}'.")
except Exception as e:
    print(f"❌ Could not create Greeting agent. Check API Key ({greeting_agent.model}). Error: {e}")

# --- Farewell Agent ---
farewell_agent = None
try:
    farewell_agent = Agent(
        # Can use the same or a different model
        model = MODEL_GEMINI_2_0_FLASH,
        # model=LiteLlm(model=MODEL_GPT_4O), # If you would like to experiment with other models
        name="farewell_agent",
        instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message. "
                    "Use the 'say_goodbye' tool when the user indicates they are leaving or ending the conversation "
                    "(e.g., using words like 'bye', 'goodbye', 'thanks bye', 'see you'). "
                    "Do not perform any other actions.",
        description="Handles simple farewells and goodbyes using the 'say_goodbye' tool.", # Crucial for delegation
        tools=[say_goodbye],
    )
    print(f"✅ Agent '{farewell_agent.name}' created using model '{farewell_agent.model}'.")
except Exception as e:
    print(f"❌ Could not create Farewell agent. Check API Key ({farewell_agent.model}). Error: {e}")

### 3\.  サブエージェントを持つルートエージェント（Weather Agent v2）を定義する

次に、 をアップグレードします。主な変更点は次のとおりです。 `weather_agent`
- `sub_agents` パラメータの追加：作成した `greeting_agent` と `farewell_agent` インスタンスを含むリストを渡します。
- `instruction` の更新：ルートエージェントにサブエージェントについて、およびタスクをいつ委任すべきかを明示的に伝えます。

**主要概念：自動委任（自動フロー）** `sub_agents` リストを提供することにより、ADKは自動委任を可能にします。ルートエージェントがユーザークエリを受信すると、そのLLMは自身の指示とツールだけでなく、各サブエージェントの `description` も考慮します。LLMがクエリがサブエージェントの記述された機能（例：「簡単な挨拶を処理する」）により適合すると判断した場合、そのターンの制御をそのサブエージェントに_移す_ための特別な内部アクションを自動的に生成します。その後、サブエージェントは独自のモデル、指示、およびツールを使用してクエリを処理します。

**ベストプラクティス：** ルートエージェントの指示が委任の決定を明確に導くようにします。サブエージェントを名前で言及し、委任が発生すべき条件を記述します。


In [ ]:
# Ensure sub-agents were created successfully before defining the root agent.
# Also ensure the original 'get_weather' tool is defined.
root_agent = None
runner_root = None # Initialize runner

if greeting_agent and farewell_agent and 'get_weather' in globals():
    # Let's use a capable Gemini model for the root agent to handle orchestration
    root_agent_model = MODEL_GEMINI_2_0_FLASH

    weather_agent_team = Agent(
        name="weather_agent_v2", # Give it a new version name
        model=root_agent_model,
        description="The main coordinator agent. Handles weather requests and delegates greetings/farewells to specialists.",
        instruction="You are the main Weather Agent coordinating a team. Your primary responsibility is to provide weather information. "
                    "Use the 'get_weather' tool ONLY for specific weather requests (e.g., 'weather in London'). "
                    "You have specialized sub-agents: "
                    "1. 'greeting_agent': Handles simple greetings like 'Hi', 'Hello'. Delegate to it for these. "
                    "2. 'farewell_agent': Handles simple farewells like 'Bye', 'See you'. Delegate to it for these. "
                    "Analyze the user's query. If it's a greeting, delegate to 'greeting_agent'. If it's a farewell, delegate to 'farewell_agent'. "
                    "If it's a weather request, handle it yourself using 'get_weather'. "
                    "For anything else, respond appropriately or state you cannot handle it.",
        tools=[get_weather], # Root agent still needs the weather tool for its core task
        # Key change: Link the sub-agents here!
        sub_agents=[greeting_agent, farewell_agent]
    )
    print(f"✅ Root Agent '{weather_agent_team.name}' created using model '{root_agent_model}' with sub-agents: {[sa.name for sa in weather_agent_team.sub_agents]}")

else:
    print("❌ Cannot create root agent because one or more sub-agents failed to initialize or 'get_weather' tool is missing.")
    if not greeting_agent: print(" - Greeting Agent is missing.")
    if not farewell_agent: print(" - Farewell Agent is missing.")
    if 'get_weather' not in globals(): print(" - get_weather function is missing.")



### 4\.  エージェントチームと対話する

専門のサブエージェントを持つルートエージェント（ - _注：この変数名が、前のコードブロック、おそらく `# @title Define the Root Agent with Sub-Agents` で定義されたものと一致していることを確認してください。そこでは `root_agent` という名前が付けられている可能性があります_）を定義したので、委任メカニズムをテストしましょう。 `weather_agent_team`
次のコードブロックは次のことを行います。

1. `async` 関数 を定義します。 `run_team_conversation`
2. この関数内で、このテスト実行専用の_新しい専用の_ `InMemorySessionService` と特定のセッション（`session_001_agent_team`）を作成します。これにより、チームのダイナミクスをテストするための会話履歴が分離されます。

3. （ルートエージェント）と専用のセッションサービスを使用するように構成された `Runner`（`runner_agent_team`）を作成します。 `weather_agent_team`
4. 更新された 関数を使用して、さまざまな種類のクエリ（挨拶、天気予報のリクエスト、別れ）を `runner_agent_team` に送信します。この特定のテストのために、ランナー、ユーザーID、およびセッションIDを明示的に渡します。 `call_agent_async`
5. 関数を直ちに実行します。 `run_team_conversation`

次のフローが期待されます。
1. 「Hello there!」というクエリが `runner_agent_team` に送られます。
2. ルートエージェント（）はそれを受け取り、その指示と `greeting_agent` の説明に基づいてタスクを委任します。 `weather_agent_team`
3. `greeting_agent` がクエリを処理し、その ツールを呼び出し、応答を生成します。 `say_hello`
4. 「What is the weather in New York?」というクエリは委任されず、ルートエージェントがその ツールを使用して直接処理します。 `get_weather`
5. 「Thanks, bye!」というクエリは `farewell_agent` に委任され、`farewell_agent` はその ツールを使用します。 `say_goodbye`




In [ ]:
import asyncio # Ensure asyncio is imported

# Ensure the root agent (e.g., 'weather_agent_team' or 'root_agent' from the previous cell) is defined.
# Ensure the call_agent_async function is defined.

# Check if the root agent variable exists before defining the conversation function
root_agent_var_name = 'root_agent' # Default name from Step 3 guide
if 'weather_agent_team' in globals(): # Check if user used this name instead
    root_agent_var_name = 'weather_agent_team'
elif 'root_agent' not in globals():
    print("⚠️ Root agent ('root_agent' or 'weather_agent_team') not found. Cannot define run_team_conversation.")
    # Assign a dummy value to prevent NameError later if the code block runs anyway
    root_agent = None # Or set a flag to prevent execution

# Only define and run if the root agent exists
if root_agent_var_name in globals() and globals()[root_agent_var_name]:
    # Define the main async function for the conversation logic.
    # The 'await' keywords INSIDE this function are necessary for async operations.
    async def run_team_conversation():
        print("\n--- Testing Agent Team Delegation ---")
        session_service = InMemorySessionService()
        APP_NAME = "weather_tutorial_agent_team"
        USER_ID = "user_1_agent_team"
        SESSION_ID = "session_001_agent_team"
        session = session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID
        )
        print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

        actual_root_agent = globals()[root_agent_var_name]
        runner_agent_team = Runner( # Or use InMemoryRunner
            agent=actual_root_agent,
            app_name=APP_NAME,
            session_service=session_service
        )
        print(f"Runner created for agent '{actual_root_agent.name}'.")

        # --- Interactions using await (correct within async def) ---
        await call_agent_async(query = "Hello there!",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)
        await call_agent_async(query = "What is the weather in New York?",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)
        await call_agent_async(query = "Thanks, bye!",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)

    # --- Execute the `run_team_conversation` async function ---
    # Choose ONE of the methods below based on your environment.
    # Note: This may require API keys for the models used!

    # METHOD 1: Direct await (Default for Notebooks/Async REPLs)
    # If your environment supports top-level await (like Colab/Jupyter notebooks),
    # it means an event loop is already running, so you can directly await the function.
    print("Attempting execution using 'await' (default for notebooks)...")
    await run_team_conversation()

    # METHOD 2: asyncio.run (For Standard Python Scripts [.py])
    # If running this code as a standard Python script from your terminal,
    # the script context is synchronous. `asyncio.run()` is needed to
    # create and manage an event loop to execute your async function.
    # To use this method:
    # 1. Comment out the `await run_team_conversation()` line above.
    # 2. Uncomment the following block:
    """
    import asyncio
    if __name__ == "__main__": # Ensures this runs only when script is executed directly
        print("Executing using 'asyncio.run()' (for standard Python scripts)...")
        try:
            # This creates an event loop, runs your async function, and closes the loop.
            asyncio.run(run_team_conversation())
        except Exception as e:
            print(f"An error occurred: {e}")
    """

else:
    # This message prints if the root agent variable wasn't found earlier
    print("\n⚠️ Skipping agent team conversation execution as the root agent was not successfully defined in a previous step.")

---

出力ログ、特に `--- Tool: ... called ---` というメッセージを注意深く見てください。次のことが確認できるはずです。
- 「Hello there!」に対して、 ツールが呼び出されました（`greeting_agent` が処理したことを示します）。 `say_hello`
- 「What is the weather in New York?」に対して、 ツールが呼び出されました（ルートエージェントが処理したことを示します）。 `get_weather`
- 「Thanks, bye!」に対して、 ツールが呼び出されました（`farewell_agent` が処理したことを示します）。 `say_goodbye`

これにより、**自動委任**が成功したことが確認できます！ルートエージェントは、その指示と `sub_agents` の `description` に導かれ、ユーザーリクエストをチーム内の適切なスペシャリストエージェントに正しくルーティングしました。
これで、複数の協調するエージェントでアプリケーションを構成しました。このモジュラー設計は、より複雑で有能なエージェントシステムを構築するための基本です。次のステップでは、セッション状態を使用して、エージェントがターンをまたいで情報を記憶できるようにします。


## ステップ3：セッション状態でメモリとパーソナライゼーションを追加する

これまでのところ、エージェントチームは委任を通じてさまざまなタスクを処理できましたが、各インタラクションは最初からやり直しでした。エージェントは、セッション内の過去の会話やユーザーの好みを記憶していません。より洗練されたコンテキスト認識型のエクスペリエンスを作成するには、エージェントに**メモリ**が必要です。ADKはこれを**セッション状態**を通じて提供します。

**セッション状態とは？**
- これは、特定のユーザーセッション（、、 によって識別される）に紐付けられたPython辞書（`session.state`）です。 `APP_NAME``USER_ID``SESSION_ID`
- そのセッション内の_複数の会話ターン_にわたって情報を永続化します。
- エージェントとツールはこの状態を読み書きできるため、詳細を記憶したり、動作を適応させたり、応答をパーソナライズしたりできます。

**エージェントが状態と対話する方法：**
1. **`ToolContext`（主要な方法）：** ツールは `ToolContext` オブジェクトを受け入れることができます（最後の引数として宣言されている場合、ADKによって自動的に提供されます）。このオブジェクトは、`tool_context.state` を介してセッション状態への直接アクセスを提供し、ツールが実行_中_に設定を読み取ったり、結果を保存したりできるようにします。
2. **`output_key`（エージェント応答の自動保存）：** `Agent` は `output_key="your_key"` で構成できます。その後、ADKはターンのエージェントの最終的なテキスト応答を `session.state["your_key"]` に自動的に保存します。

**このステップでは、次の方法でWeather Botチームを強化します。**
1. 分離された状態で状態をデモンストレーションするために、**新しい** `InMemorySessionService` を使用します。
2. `temperature_unit` のユーザー設定でセッション状態を初期化します。
3. `ToolContext` を介してこの設定を読み取り、出力形式（摂氏/華氏）を調整する、状態認識型の天気ツール（）のバージョンを作成します。 `get_weather_stateful`
4. このステートフルツールを使用するようにルートエージェントを更新し、最終的な天気予報をセッション状態に自動的に保存するように `output_key` で構成します。
5. 会話を実行して、初期状態がツールにどのように影響するか、手動での状態変更がその後の動作をどのように変更するか、そして `output_key` がエージェントの応答をどのように永続化するかを観察します。


### 1\.  新しいセッションサービスと状態を初期化する
以前のステップからの干渉なしに状態管理を明確に示すために、新しい `InMemorySessionService` をインスタンス化します。また、ユーザーの希望する温度単位を定義する初期状態を持つセッションも作成します。


In [ ]:
# Import necessary session components
from google.adk.sessions import InMemorySessionService

# Create a NEW session service instance for this state demonstration
session_service_stateful = InMemorySessionService()
print("✅ New InMemorySessionService created for state demonstration.")

# Define a NEW session ID for this part of the tutorial
SESSION_ID_STATEFUL = "session_state_demo_001"
USER_ID_STATEFUL = "user_state_demo"

# Define initial state data - user prefers Celsius initially
initial_state = {
    "user_preference_temperature_unit": "Celsius"
}

# Create the session, providing the initial state
session_stateful = session_service_stateful.create_session(
    app_name=APP_NAME, # Use the consistent app name
    user_id=USER_ID_STATEFUL,
    session_id=SESSION_ID_STATEFUL,
    state=initial_state # <<< Initialize state during creation
)
print(f"✅ Session '{SESSION_ID_STATEFUL}' created for user '{USER_ID_STATEFUL}'.")

# Verify the initial state was set correctly
retrieved_session = session_service_stateful.get_session(app_name=APP_NAME,
                                                         user_id=USER_ID_STATEFUL,
                                                         session_id = SESSION_ID_STATEFUL)
print("\n--- Initial Session State ---")
if retrieved_session:
    print(retrieved_session.state)
else:
    print("Error: Could not retrieve session.")

### 2\.  状態を意識した天気予報ツールを作成する (`get_weather_stateful`)

次に、天気予報ツールの新しいバージョンを作成します。その主な機能は `tool_context: ToolContext` を受け入れることであり、これにより `tool_context.state` にアクセスできるようになります。このツールは `user_preference_temperature_unit` を読み取り、それに応じて温度の形式を調整します。

- **キーコンセプト： `ToolContext`** このオブジェクトは、ツールロジックがセッションのコンテキスト（状態変数の読み取りと書き込みを含む）と対話できるようにするブリッジです。ツール関数の最後のパラメータとして定義されている場合、ADKはそれを自動的に挿入します。

- **ベストプラクティス：** 状態から読み取る場合は、`dictionary.get('key', default_value)` を使用して、キーがまだ存在しない可能性があるケースを処理し、ツールがクラッシュしないようにします。


In [ ]:
from google.adk.tools.tool_context import ToolContext

def get_weather_stateful(city: str, tool_context: ToolContext) -> dict:
    """Retrieves weather, converts temp unit based on session state."""
    print(f"--- Tool: get_weather_stateful called for {city} ---")

    # --- Read preference from state ---
    preferred_unit = tool_context.state.get("user_preference_temperature_unit", "Celsius") # Default to Celsius
    print(f"--- Tool: Reading state 'user_preference_temperature_unit': {preferred_unit} ---")

    city_normalized = city.lower().replace(" ", "")

    # Mock weather data (always stored in Celsius internally)
    mock_weather_db = {
        "newyork": {"temp_c": 25, "condition": "sunny"},
        "london": {"temp_c": 15, "condition": "cloudy"},
        "tokyo": {"temp_c": 18, "condition": "light rain"},
    }

    if city_normalized in mock_weather_db:
        data = mock_weather_db[city_normalized]
        temp_c = data["temp_c"]
        condition = data["condition"]

        # Format temperature based on state preference
        if preferred_unit == "Fahrenheit":
            temp_value = (temp_c * 9/5) + 32 # Calculate Fahrenheit
            temp_unit = "°F"
        else: # Default to Celsius
            temp_value = temp_c
            temp_unit = "°C"

        report = f"The weather in {city.capitalize()} is {condition} with a temperature of {temp_value:.0f}{temp_unit}."
        result = {"status": "success", "report": report}
        print(f"--- Tool: Generated report in {preferred_unit}. Result: {result} ---")

        # Example of writing back to state (optional for this tool)
        tool_context.state["last_city_checked_stateful"] = city
        print(f"--- Tool: Updated state 'last_city_checked_stateful': {city} ---")

        return result
    else:
        # Handle city not found
        error_msg = f"Sorry, I don't have weather information for '{city}'."
        print(f"--- Tool: City '{city}' not found. ---")
        return {"status": "error", "error_message": error_msg}

print("✅ State-aware 'get_weather_stateful' tool defined.")


### 3\.  サブエージェントを再定義し、ルートエージェントを更新する

このステップが自己完結型であり、正しく構築されることを保証するために、まずステップ3とまったく同じように `greeting_agent` と `farewell_agent` を再定義します。次に、新しいルートエージェント（`weather_agent_v4_stateful`）を定義します。

- 新しい ツールを使用します。 `get_weather_stateful`
- 委任のために挨拶と別れのサブエージェントを含みます。
- **重要なこと**として、`output_key="last_weather_report"` を設定します。これにより、最終的な天気予報の応答がセッション状態に自動的に保存されます。


In [ ]:
# Ensure necessary imports: Agent, LiteLlm, Runner
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm
from google.adk.runners import Runner
# Ensure tools 'say_hello', 'say_goodbye' are defined (from Step 3)
# Ensure model constants MODEL_GPT_4O, MODEL_GEMINI_2_0_FLASH etc. are defined

# --- Redefine Greeting Agent (from Step 3) ---
greeting_agent = None
try:
    greeting_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="greeting_agent",
        instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting using the 'say_hello' tool. Do nothing else.",
        description="Handles simple greetings and hellos using the 'say_hello' tool.",
        tools=[say_hello],
    )
    print(f"✅ Agent '{greeting_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Greeting agent. Error: {e}")

# --- Redefine Farewell Agent (from Step 3) ---
farewell_agent = None
try:
    farewell_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="farewell_agent",
        instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message using the 'say_goodbye' tool. Do not perform any other actions.",
        description="Handles simple farewells and goodbyes using the 'say_goodbye' tool.",
        tools=[say_goodbye],
    )
    print(f"✅ Agent '{farewell_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Farewell agent. Error: {e}")

# --- Define the Updated Root Agent ---
root_agent_stateful = None
runner_root_stateful = None # Initialize runner

# Check prerequisites before creating the root agent
if greeting_agent and farewell_agent and 'get_weather_stateful' in globals():

    root_agent_model = MODEL_GEMINI_2_0_FLASH # Choose orchestration model

    root_agent_stateful = Agent(
        name="weather_agent_v4_stateful", # New version name
        model=root_agent_model,
        description="Main agent: Provides weather (state-aware unit), delegates greetings/farewells, saves report to state.",
        instruction="You are the main Weather Agent. Your job is to provide weather using 'get_weather_stateful'. "
                    "The tool will format the temperature based on user preference stored in state. "
                    "Delegate simple greetings to 'greeting_agent' and farewells to 'farewell_agent'. "
                    "Handle only weather requests, greetings, and farewells.",
        tools=[get_weather_stateful], # Use the state-aware tool
        sub_agents=[greeting_agent, farewell_agent], # Include sub-agents
        output_key="last_weather_report" # <<< Auto-save agent's final weather response
    )
    print(f"✅ Root Agent '{root_agent_stateful.name}' created using stateful tool and output_key.")

    # --- Create Runner for this Root Agent & NEW Session Service ---
    runner_root_stateful = Runner(
        agent=root_agent_stateful,
        app_name=APP_NAME,
        session_service=session_service_stateful # Use the NEW stateful session service
    )
    print(f"✅ Runner created for stateful root agent '{runner_root_stateful.agent.name}' using stateful session service.")

else:
    print("❌ Cannot create stateful root agent. Prerequisites missing.")
    if not greeting_agent: print(" - greeting_agent definition missing.")
    if not farewell_agent: print(" - farewell_agent definition missing.")
    if 'get_weather_stateful' not in globals(): print(" - get_weather_stateful tool missing.")


### 4\.  対話し、状態フローをテストする

それでは、`runner_root_stateful`（ステートフルなエージェントと に関連付けられている）を使用して、状態の相互作用をテストするように設計された会話を実行しましょう。以前に定義した 関数を使用し、正しいランナー、ユーザーID（）、セッションID（）を渡すようにします。 `session_service_stateful``call_agent_async``USER_ID_STATEFUL``SESSION_ID_STATEFUL`
会話の流れは次のようになります。

1. **天気の確認（ロンドン）：** ツールは、セクション1で初期化されたセッション状態から初期の「摂氏」設定を読み取る必要があります。ルートエージェントの最終応答（摂氏での天気予報）は、`output_key` 設定を介して `state['last_weather_report']` に保存される必要があります。 `get_weather_stateful`
2. **状態を手動で更新する：** `InMemorySessionService` インスタンス（）内に保存されている状態を_直接変更_します。
    - **なぜ直接変更するのか？** `session_service.get_session()` メソッドはセッションの_コピー_を返します。そのコピーを変更しても、後続のエージェント実行で使用される状態には影響しません。この `InMemorySessionService` を使用したテストシナリオでは、内部の 辞書にアクセスして、`user_preference_temperature_unit` の_実際に_保存されている状態値を「華氏」に変更します。_注：実際のアプリケーションでは、状態の変更は通常、ツールまたはエージェントロジックが `EventActions(state_delta=...)` を返すことによってトリガーされ、直接の手動更新ではありません。_ `sessions`

`session_service_stateful`
3. **再度天気をチェックする（ニューヨーク）：** ツールは、状態から更新された「華氏」の設定を読み取り、それに応じて温度を変換する必要があります。ルートエージェントの_新しい_応答（華氏での天気）は、`output_key` により `state['last_weather_report']` の以前の値を上書きします。 `get_weather_stateful`
4. **エージェントに挨拶する：** `greeting_agent` への委任が、ステートフルな操作と並行して引き続き正しく機能することを確認します。このインタラクションは、この特定のシーケンスで `output_key` によって保存される_最後_の応答になります。
5. **最終状態を検査する：** 会話の後、最後にセッションを取得し（コピーを取得）、その状態を出力して、`user_preference_temperature_unit` が実際に「華氏」であることを確認し、`output_key` によって保存された最終値（この実行では挨拶になります）を観察し、ツールによって書き込まれた `last_city_checked_stateful` 値を確認します。



In [ ]:
import asyncio # Ensure asyncio is imported

# Ensure the stateful runner (runner_root_stateful) is available from the previous cell
# Ensure call_agent_async, USER_ID_STATEFUL, SESSION_ID_STATEFUL, APP_NAME are defined

if 'runner_root_stateful' in globals() and runner_root_stateful:
    # Define the main async function for the stateful conversation logic.
    # The 'await' keywords INSIDE this function are necessary for async operations.
    async def run_stateful_conversation():
        print("\n--- Testing State: Temp Unit Conversion & output_key ---")

        # 1. Check weather (Uses initial state: Celsius)
        print("--- Turn 1: Requesting weather in London (expect Celsius) ---")
        await call_agent_async(query= "What's the weather in London?",
                               runner=runner_root_stateful,
                               user_id=USER_ID_STATEFUL,
                               session_id=SESSION_ID_STATEFUL
                              )

        # 2. Manually update state preference to Fahrenheit - DIRECTLY MODIFY STORAGE
        print("\n--- Manually Updating State: Setting unit to Fahrenheit ---")
        try:
            # Access the internal storage directly - THIS IS SPECIFIC TO InMemorySessionService for testing
            # NOTE: In production with persistent services (Database, VertexAI), you would
            # typically update state via agent actions or specific service APIs if available,
            # not by direct manipulation of internal storage.
            stored_session = session_service_stateful.sessions[APP_NAME][USER_ID_STATEFUL][SESSION_ID_STATEFUL]
            stored_session.state["user_preference_temperature_unit"] = "Fahrenheit"
            # Optional: You might want to update the timestamp as well if any logic depends on it
            # import time
            # stored_session.last_update_time = time.time()
            print(f"--- Stored session state updated. Current 'user_preference_temperature_unit': {stored_session.state.get('user_preference_temperature_unit', 'Not Set')} ---") # Added .get for safety
        except KeyError:
            print(f"--- Error: Could not retrieve session '{SESSION_ID_STATEFUL}' from internal storage for user '{USER_ID_STATEFUL}' in app '{APP_NAME}' to update state. Check IDs and if session was created. ---")
        except Exception as e:
             print(f"--- Error updating internal session state: {e} ---")

        # 3. Check weather again (Tool should now use Fahrenheit)
        # This will also update 'last_weather_report' via output_key
        print("\n--- Turn 2: Requesting weather in New York (expect Fahrenheit) ---")
        await call_agent_async(query= "Tell me the weather in New York.",
                               runner=runner_root_stateful,
                               user_id=USER_ID_STATEFUL,
                               session_id=SESSION_ID_STATEFUL
                              )

        # 4. Test basic delegation (should still work)
        # This will update 'last_weather_report' again, overwriting the NY weather report
        print("\n--- Turn 3: Sending a greeting ---")
        await call_agent_async(query= "Hi!",
                               runner=runner_root_stateful,
                               user_id=USER_ID_STATEFUL,
                               session_id=SESSION_ID_STATEFUL
                              )

    # --- Execute the `run_stateful_conversation` async function ---
    # Choose ONE of the methods below based on your environment.

    # METHOD 1: Direct await (Default for Notebooks/Async REPLs)
    # If your environment supports top-level await (like Colab/Jupyter notebooks),
    # it means an event loop is already running, so you can directly await the function.
    print("Attempting execution using 'await' (default for notebooks)...")
    await run_stateful_conversation()

    # METHOD 2: asyncio.run (For Standard Python Scripts [.py])
    # If running this code as a standard Python script from your terminal,
    # the script context is synchronous. `asyncio.run()` is needed to
    # create and manage an event loop to execute your async function.
    # To use this method:
    # 1. Comment out the `await run_stateful_conversation()` line above.
    # 2. Uncomment the following block:
    """
    import asyncio
    if __name__ == "__main__": # Ensures this runs only when script is executed directly
        print("Executing using 'asyncio.run()' (for standard Python scripts)...")
        try:
            # This creates an event loop, runs your async function, and closes the loop.
            asyncio.run(run_stateful_conversation())
        except Exception as e:
            print(f"An error occurred: {e}")
    """

    # --- Inspect final session state after the conversation ---
    # This block runs after either execution method completes.
    print("\n--- Inspecting Final Session State ---")
    final_session = session_service_stateful.get_session(app_name=APP_NAME,
                                                         user_id= USER_ID_STATEFUL,
                                                         session_id=SESSION_ID_STATEFUL)
    if final_session:
        # Use .get() for safer access to potentially missing keys
        print(f"Final Preference: {final_session.state.get('user_preference_temperature_unit', 'Not Set')}")
        print(f"Final Last Weather Report (from output_key): {final_session.state.get('last_weather_report', 'Not Set')}")
        print(f"Final Last City Checked (by tool): {final_session.state.get('last_city_checked_stateful', 'Not Set')}")
        # Print full state for detailed view
        # print(f"Full State Dict: {final_session.state.as_dict()}") # Use as_dict() for clarity
    else:
        print("\n❌ Error: Could not retrieve final session state.")

else:
    print("\n⚠️ Skipping state test conversation. Stateful root agent runner ('runner_root_stateful') is not available.")

---

会話のフローと最終的なセッション状態のプリントアウトを確認することで、以下を確認できます。
- **状態の読み取り：** 天気予報ツール（）は、状態から `user_preference_temperature_unit` を正しく読み取り、最初はロンドンに対して「摂氏」を使用しました。 `get_weather_stateful`
- **状態の更新：** 直接的な変更により、保存されていた設定が「華氏」に正常に変更されました。
- **状態の読み取り（更新済み）：** ニューヨークの天気を尋ねられた際、ツールはその後「華氏」を読み取り、変換を実行しました。
- **ツールの状態書き込み：** ツールは、`tool_context.state` を介して `last_city_checked_stateful`（2回目の天気チェック後の「ニューヨーク」）を状態に正常に書き込みました。
- **委任：** 「Hi!」に対する `greeting_agent` への委任は、状態変更後も正しく機能しました。
- **`output_key`：** `output_key="last_weather_report"` は、ルートエージェントが最終的に応答した_各ターン_において、ルートエージェントの_最終_応答を正常に保存しました。このシーケンスでは、最後の応答は挨拶（「Hello, there!」）であったため、それが状態キーの天気予報を上書きしました。
- **最終状態：** 最終チェックにより、設定が「華氏」として永続化されていることが確認されます。

これで、`ToolContext` を使用してエージェントの動作をパーソナライズするためのセッション状態の統合、`InMemorySessionService` のテストのための状態の手動操作、そして `output_key` がエージェントの最後の応答を状態に保存するための簡単なメカニズムをどのように提供するかを観察することに成功しました。この状態管理の基礎的な理解は、次のステップでコールバックを使用して安全ガードレールを実装する際に重要となります。

---

## ステップ4：安全性の追加 - `before_model_callback` を使用した入力ガードレール

私たちのエージェントチームは、より能力を高め、好みを記憶し、ツールを効果的に使用できるようになりました。しかし、現実世界のシナリオでは、潜在的に問題のあるリクエストがコアの大規模言語モデル（LLM）に到達する_前_に、エージェントの動作を制御するための安全メカニズムが必要になることがよくあります。

ADK は**コールバック**を提供します。これは、エージェントの実行ライフサイクルの特定のポイントにフックできる関数です。`before_model_callback` は、入力の安全性にとって特に便利です。

**`before_model_callback` とは？**
- これは、ADK がエージェントがコンパイルされたリクエスト（会話履歴、指示、最新のユーザーメッセージを含む）を基礎となる LLM に送信する_直前_に実行する、ユーザーが定義する Python 関数です。
- **目的：** リクエストを検査し、必要に応じて変更するか、事前定義されたルールに基づいて完全にブロックします。

**一般的なユースケース：**
- **入力の検証/フィルタリング：** ユーザー入力が基準を満たしているか、許可されていないコンテンツ（PII やキーワードなど）が含まれていないかを確認します。
- **ガードレール：** 有害な、トピック外の、またはポリシーに違反するリクエストが LLM によって処理されるのを防ぎます。
- **動的なプロンプトの変更：** 送信する直前に、タイムリーな情報（セッション状態からの情報など）を LLM リクエストコンテキストに追加します。

**仕組み：**
1. `callback_context: CallbackContext` と `llm_request: LlmRequest` を受け入れる関数を定義します。
    - `callback_context`：エージェント情報、セッション状態（`callback_context.state`）などへのアクセスを提供します。
    - `llm_request`：LLM 向けの完全なペイロード（`contents`、`config`）を含みます。

2. 関数内で：
    - **検査：** `llm_request.contents`（特に最後のユーザーメッセージ）を調べます。
    - **変更（注意して使用）：** `llm_request` の一部を_変更できます_。
    - **ブロック（ガードレール）：** `LlmResponse` オブジェクトを返します。ADK はこの応答をすぐに送り返し、そのターンの LLM 呼び出しを_スキップ_します。
    - **許可：** `None` を返します。ADK は（変更された可能性のある）リクエストで LLM の呼び出しに進みます。

**このステップでは、次のことを行います。**
1. 特定のキーワード（"BLOCK"）についてユーザーの入力をチェックする `before_model_callback` 関数（）を定義します。 `block_keyword_guardrail`
2. ステップ4のステートフルなルートエージェント（`weather_agent_v4_stateful`）を更新して、このコールバックを使用するようにします。
3. この更新されたエージェントに関連付けられた新しいランナーを作成しますが、状態の継続性を維持するために_同じステートフルなセッションサービス_を使用します。
4. 通常のリクエストとキーワードを含むリクエストの両方を送信して、ガードレールをテストします。


### 1\.  ガードレールコールバック関数を定義する

この関数は、`llm_request` コンテンツ内の最後のユーザーメッセージを検査します。「BLOCK」（大文字・小文字を区別しない）が見つかった場合、フローをブロックするために `LlmResponse` を構築して返します。それ以外の場合は `None` を返します。


In [ ]:
# Ensure necessary imports are available
from google.adk.agents.callback_context import CallbackContext
from google.adk.models.llm_request import LlmRequest
from google.adk.models.llm_response import LlmResponse
from google.genai import types # For creating response content
from typing import Optional

def block_keyword_guardrail(
    callback_context: CallbackContext, llm_request: LlmRequest
) -> Optional[LlmResponse]:
    """
    Inspects the latest user message for 'BLOCK'. If found, blocks the LLM call
    and returns a predefined LlmResponse. Otherwise, returns None to proceed.
    """
    agent_name = callback_context.agent_name # Get the name of the agent whose model call is being intercepted
    print(f"--- Callback: block_keyword_guardrail running for agent: {agent_name} ---")

    # Extract the text from the latest user message in the request history
    last_user_message_text = ""
    if llm_request.contents:
        # Find the most recent message with role 'user'
        for content in reversed(llm_request.contents):
            if content.role == 'user' and content.parts:
                # Assuming text is in the first part for simplicity
                if content.parts[0].text:
                    last_user_message_text = content.parts[0].text
                    break # Found the last user message text

    print(f"--- Callback: Inspecting last user message: '{last_user_message_text[:100]}...' ---") # Log first 100 chars

    # --- Guardrail Logic ---
    keyword_to_block = "BLOCK"
    if keyword_to_block in last_user_message_text.upper(): # Case-insensitive check
        print(f"--- Callback: Found '{keyword_to_block}'. Blocking LLM call! ---")
        # Optionally, set a flag in state to record the block event
        callback_context.state["guardrail_block_keyword_triggered"] = True
        print(f"--- Callback: Set state 'guardrail_block_keyword_triggered': True ---")

        # Construct and return an LlmResponse to stop the flow and send this back instead
        return LlmResponse(
            content=types.Content(
                role="model", # Mimic a response from the agent's perspective
                parts=[types.Part(text=f"I cannot process this request because it contains the blocked keyword '{keyword_to_block}'.")],
            )
            # Note: You could also set an error_message field here if needed
        )
    else:
        # Keyword not found, allow the request to proceed to the LLM
        print(f"--- Callback: Keyword not found. Allowing LLM call for {agent_name}. ---")
        return None # Returning None signals ADK to continue normally

print("✅ block_keyword_guardrail function defined.")


### 2\. コールバックを使用するようにルートエージェントを更新する**

ルートエージェントを再定義し、`before_model_callback` パラメータを追加して、新しいガードレール関数を指定します。明確にするために、新しいバージョン名を付けます。
_重要:_ ルートエージェントの定義がすべてのコンポーネントにアクセスできるように、サブエージェント（`greeting_agent`、`farewell_agent`）とステートフルツール（）が以前のステップからまだ利用できない場合は、このコンテキスト内で再定義する必要があります。 `get_weather_stateful`


In [ ]:
# --- Redefine Sub-Agents (Ensures they exist in this context) ---
greeting_agent = None
try:
    # Use a defined model constant
    greeting_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="greeting_agent", # Keep original name for consistency
        instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting using the 'say_hello' tool. Do nothing else.",
        description="Handles simple greetings and hellos using the 'say_hello' tool.",
        tools=[say_hello],
    )
    print(f"✅ Sub-Agent '{greeting_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Greeting agent. Check Model/API Key ({greeting_agent.model}). Error: {e}")

farewell_agent = None
try:
    # Use a defined model constant
    farewell_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="farewell_agent", # Keep original name
        instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message using the 'say_goodbye' tool. Do not perform any other actions.",
        description="Handles simple farewells and goodbyes using the 'say_goodbye' tool.",
        tools=[say_goodbye],
    )
    print(f"✅ Sub-Agent '{farewell_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Farewell agent. Check Model/API Key ({farewell_agent.model}). Error: {e}")


# --- Define the Root Agent with the Callback ---
root_agent_model_guardrail = None
runner_root_model_guardrail = None

# Check all components before proceeding
if greeting_agent and farewell_agent and 'get_weather_stateful' in globals() and 'block_keyword_guardrail' in globals():

    # Use a defined model constant
    root_agent_model = MODEL_GEMINI_2_0_FLASH

    root_agent_model_guardrail = Agent(
        name="weather_agent_v5_model_guardrail", # New version name for clarity
        model=root_agent_model,
        description="Main agent: Handles weather, delegates greetings/farewells, includes input keyword guardrail.",
        instruction="You are the main Weather Agent. Provide weather using 'get_weather_stateful'. "
                    "Delegate simple greetings to 'greeting_agent' and farewells to 'farewell_agent'. "
                    "Handle only weather requests, greetings, and farewells.",
        tools=[get_weather],
        sub_agents=[greeting_agent, farewell_agent], # Reference the redefined sub-agents
        output_key="last_weather_report", # Keep output_key from Step 4
        before_model_callback=block_keyword_guardrail # <<< Assign the guardrail callback
    )
    print(f"✅ Root Agent '{root_agent_model_guardrail.name}' created with before_model_callback.")

    # --- Create Runner for this Agent, Using SAME Stateful Session Service ---
    # Ensure session_service_stateful exists from Step 4
    if 'session_service_stateful' in globals():
        runner_root_model_guardrail = Runner(
            agent=root_agent_model_guardrail,
            app_name=APP_NAME, # Use consistent APP_NAME
            session_service=session_service_stateful # <<< Use the service from Step 4
        )
        print(f"✅ Runner created for guardrail agent '{runner_root_model_guardrail.agent.name}', using stateful session service.")
    else:
        print("❌ Cannot create runner. 'session_service_stateful' from Step 4 is missing.")

else:
    print("❌ Cannot create root agent with model guardrail. One or more prerequisites are missing or failed initialization:")
    if not greeting_agent: print("   - Greeting Agent")
    if not farewell_agent: print("   - Farewell Agent")
    if 'get_weather_stateful' not in globals(): print("   - 'get_weather_stateful' tool")
    if 'block_keyword_guardrail' not in globals(): print("   - 'block_keyword_guardrail' callback")

### 3\.  対話してガードレールをテストする**

ガードレールの動作をテストしましょう。ステップ4と同じ_セッション_（）を使用して、これらの変更間で状態が持続することを示します。 `SESSION_ID_STATEFUL`
1. 通常の天気予報リクエストを送信します（ガードレールを通過して実行されるはずです）。
2. 「BLOCK」を含むリクエストを送信します（コールバックによってインターセプトされるはずです）。
3. 挨拶を送信します（ルートエージェントのガードレールを通過し、委任され、正常に実行されるはずです）


In [ ]:
import asyncio # Ensure asyncio is imported

# Ensure the runner for the guardrail agent is available
if 'runner_root_model_guardrail' in globals() and runner_root_model_guardrail:
    # Define the main async function for the guardrail test conversation.
    # The 'await' keywords INSIDE this function are necessary for async operations.
    async def run_guardrail_test_conversation():
        print("\n--- Testing Model Input Guardrail ---")

        # Use the runner for the agent with the callback and the existing stateful session ID
        # Define a helper lambda for cleaner interaction calls
        interaction_func = lambda query: call_agent_async(query,
                                                         runner_root_model_guardrail,
                                                         USER_ID_STATEFUL, # Use existing user ID
                                                         SESSION_ID_STATEFUL # Use existing session ID
                                                        )
        # 1. Normal request (Callback allows, should use Fahrenheit from previous state change)
        print("--- Turn 1: Requesting weather in London (expect allowed, Fahrenheit) ---")
        await interaction_func("What is the weather in London?")

        # 2. Request containing the blocked keyword (Callback intercepts)
        print("\n--- Turn 2: Requesting with blocked keyword (expect blocked) ---")
        await interaction_func("BLOCK the request for weather in Tokyo") # Callback should catch "BLOCK"

        # 3. Normal greeting (Callback allows root agent, delegation happens)
        print("\n--- Turn 3: Sending a greeting (expect allowed) ---")
        await interaction_func("Hello again")

    # --- Execute the `run_guardrail_test_conversation` async function ---
    # Choose ONE of the methods below based on your environment.

    # METHOD 1: Direct await (Default for Notebooks/Async REPLs)
    # If your environment supports top-level await (like Colab/Jupyter notebooks),
    # it means an event loop is already running, so you can directly await the function.
    print("Attempting execution using 'await' (default for notebooks)...")
    await run_guardrail_test_conversation()

    # METHOD 2: asyncio.run (For Standard Python Scripts [.py])
    # If running this code as a standard Python script from your terminal,
    # the script context is synchronous. `asyncio.run()` is needed to
    # create and manage an event loop to execute your async function.
    # To use this method:
    # 1. Comment out the `await run_guardrail_test_conversation()` line above.
    # 2. Uncomment the following block:
    """
    import asyncio
    if __name__ == "__main__": # Ensures this runs only when script is executed directly
        print("Executing using 'asyncio.run()' (for standard Python scripts)...")
        try:
            # This creates an event loop, runs your async function, and closes the loop.
            asyncio.run(run_guardrail_test_conversation())
        except Exception as e:
            print(f"An error occurred: {e}")
    """

    # --- Inspect final session state after the conversation ---
    # This block runs after either execution method completes.
    # Optional: Check state for the trigger flag set by the callback
    print("\n--- Inspecting Final Session State (After Guardrail Test) ---")
    # Use the session service instance associated with this stateful session
    final_session = session_service_stateful.get_session(app_name=APP_NAME,
                                                         user_id=USER_ID_STATEFUL,
                                                         session_id=SESSION_ID_STATEFUL)
    if final_session:
        # Use .get() for safer access
        print(f"Guardrail Triggered Flag: {final_session.state.get('guardrail_block_keyword_triggered', 'Not Set (or False)')}")
        print(f"Last Weather Report: {final_session.state.get('last_weather_report', 'Not Set')}") # Should be London weather if successful
        print(f"Temperature Unit: {final_session.state.get('user_preference_temperature_unit', 'Not Set')}") # Should be Fahrenheit
        # print(f"Full State Dict: {final_session.state.as_dict()}") # For detailed view
    else:
        print("\n❌ Error: Could not retrieve final session state.")

else:
    print("\n⚠️ Skipping model guardrail test. Runner ('runner_root_model_guardrail') is not available.")

---

実行フローを観察してください。

1. **ロンドンの天気：** コールバックは `weather_agent_v5_model_guardrail` に対して実行され、メッセージを検査し、「Keyword not found. Allowing LLM call.」と出力して `None` を返します。エージェントは処理を続行し、 ツール（ステップ4の状態変更による「華氏」の優先設定を使用）を呼び出し、天気を返します。この応答は `output_key` を介して `last_weather_report` を更新します。 `get_weather_stateful`
2. **BLOCK リクエスト：** コールバックは `weather_agent_v5_model_guardrail` に対して再度実行され、メッセージを検査し、「BLOCK」を見つけて「Blocking LLM call!」と出力し、状態フラグを設定して、事前定義された `LlmResponse` を返します。このターンでは、エージェントの基礎となる LLM は_呼び出されません_。ユーザーにはコールバックのブロッキングメッセージが表示されます。
3. **再びこんにちは：** コールバックは `weather_agent_v5_model_guardrail` に対して実行され、リクエストを許可します。ルートエージェントは次に `greeting_agent` に委任します。_注意：ルートエージェントで定義された `before_model_callback` は、サブエージェントには自動的に適用されません。_ `greeting_agent` は正常に処理を進め、その ツールを呼び出し、挨拶を返します。 `say_hello`

入力安全レイヤーの実装に成功しました！ `before_model_callback` は、高価な、または潜在的に危険な LLM 呼び出しが行われる_前_に、ルールを適用し、エージェントの動作を制御するための強力なメカニズムを提供します。次に、同様の概念を適用して、ツール自体の使用に関するガードレールを追加します。


## ステップ5：安全性の追加 - ツール引数ガードレール（`before_tool_callback`）
ステップ4では、ユーザー入力がLLMに到達する_前_に検査し、潜在的にブロックするガードレールを追加しました。次に、LLMがツールの使用を決定した後、そのツールが実際に実行される_前_に、別の制御レイヤーを追加します。これは、LLMがツールに渡そうとしている_引数_を検証するのに役立ちます。
ADKは、この正確な目的のために `before_tool_callback` を提供します。
**`before_tool_callback` とは何か？**
- これは、LLMがその使用を要求し、引数を決定した後、特定のツール関数が実行される直前に実行されるPython関数です。
- **目的：** ツール引数の検証、特定の入力に基づくツール実行の防止、引数の動的な変更、またはリソース使用ポリシーの強制。

**一般的なユースケース：**
- **引数の検証：** LLMによって提供された引数が有効であるか、許容範囲内であるか、または期待される形式に準拠しているかを確認します。
- **リソース保護：** コストがかかる可能性のある入力、制限されたデータへのアクセス、または望ましくない副作用（特定のパラメータに対するAPI呼び出しのブロックなど）を引き起こす可能性のある入力でツールが呼び出されるのを防ぎます。
- **動的な引数変更：** ツールが実行される前に、セッション状態またはその他のコンテキスト情報に基づいて引数を調整します。

**仕組み：**
1. `tool: BaseTool`、`args: Dict[str, Any]`、および `tool_context: ToolContext` を受け入れる関数を定義します。
    - ：呼び出されようとしているツールオブジェクト（`tool.name` を検査）。 `tool`
    - `args`：LLMがツール用に生成した引数の辞書。
    - `tool_context`：セッション状態（`tool_context.state`）、エージェント情報などへのアクセスを提供します。

2. 関数内：
    - **検査：** `tool.name` と `args` 辞書を調べます。
    - **変更：** `args` 辞書内の値を_直接_変更します。`None` を返すと、ツールはこれらの変更された引数で実行されます。
    - **ブロック/オーバーライド（ガードレール）：** **辞書**を返します。ADKはこの辞書をツール呼び出しの_結果_として扱い、元のツール関数の実行を完全に_スキップ_します。辞書は、理想的にはブロックしているツールの期待される戻り形式と一致する必要があります。
    - **許可：** `None` を返します。ADKは、（潜在的に変更された）引数で実際のツール関数の実行に進みます。

**このステップでは、以下を行います。**
1. ツールが都市「Paris」で呼び出されたかどうかを具体的にチェックする `before_tool_callback` 関数（）を定義します。 `get_weather_stateful``block_paris_tool_guardrail`
2. 「Paris」が検出された場合、コールバックはツールをブロックし、カスタムエラー辞書を返します。
3. ルートエージェント（`weather_agent_v6_tool_guardrail`）を更新して、`before_model_callback` とこの新しい `before_tool_callback` の_両方_を含めます。
4. 同じステートフルセッションサービスを使用して、このエージェント用の新しいランナーを作成します。
5. 許可された都市とブロックされた都市（「Paris」）の天気を要求して、フローをテストします。


### 1\.  ツールガードレールコールバック関数を定義する**

この関数は ツールを対象とします。`city` 引数をチェックします。「Paris」の場合、ツール自身のエラー応答に似たエラー辞書を返します。それ以外の場合は、`None` を返すことによってツールの実行を許可します。 `get_weather_stateful`


In [ ]:
# Ensure necessary imports are available
from google.adk.tools.base_tool import BaseTool
from google.adk.tools.tool_context import ToolContext
from typing import Optional, Dict, Any # For type hints

def block_paris_tool_guardrail(
    tool: BaseTool, args: Dict[str, Any], tool_context: ToolContext
) -> Optional[Dict]:
    """
    Checks if 'get_weather_stateful' is called for 'Paris'.
    If so, blocks the tool execution and returns a specific error dictionary.
    Otherwise, allows the tool call to proceed by returning None.
    """
    tool_name = tool.name
    agent_name = tool_context.agent_name # Agent attempting the tool call
    print(f"--- Callback: block_paris_tool_guardrail running for tool '{tool_name}' in agent '{agent_name}' ---")
    print(f"--- Callback: Inspecting args: {args} ---")

    # --- Guardrail Logic ---
    target_tool_name = "get_weather_stateful" # Match the function name used by FunctionTool
    blocked_city = "paris"

    # Check if it's the correct tool and the city argument matches the blocked city
    if tool_name == target_tool_name:
        city_argument = args.get("city", "") # Safely get the 'city' argument
        if city_argument and city_argument.lower() == blocked_city:
            print(f"--- Callback: Detected blocked city '{city_argument}'. Blocking tool execution! ---")
            # Optionally update state
            tool_context.state["guardrail_tool_block_triggered"] = True
            print(f"--- Callback: Set state 'guardrail_tool_block_triggered': True ---")

            # Return a dictionary matching the tool's expected output format for errors
            # This dictionary becomes the tool's result, skipping the actual tool run.
            return {
                "status": "error",
                "error_message": f"Policy restriction: Weather checks for '{city_argument.capitalize()}' are currently disabled by a tool guardrail."
            }
        else:
             print(f"--- Callback: City '{city_argument}' is allowed for tool '{tool_name}'. ---")
    else:
        print(f"--- Callback: Tool '{tool_name}' is not the target tool. Allowing. ---")


    # If the checks above didn't return a dictionary, allow the tool to execute
    print(f"--- Callback: Allowing tool '{tool_name}' to proceed. ---")
    return None # Returning None allows the actual tool function to run

print("✅ block_paris_tool_guardrail function defined.")



### 2\.  両方のコールバックを使用するようにルートエージェントを更新する**

ルートエージェントを再度（`weather_agent_v6_tool_guardrail`）定義し、今回はステップ5の `before_model_callback` と共に `before_tool_callback` パラメータを追加します。
_自己完結型実行に関する注意：_ ステップ5と同様に、このエージェントを定義する前に、すべての前提条件（サブエージェント、ツール、`before_model_callback`）が定義されているか、実行コンテキストで使用可能であることを確認してください。


In [ ]:
# --- Ensure Prerequisites are Defined ---
# (Include or ensure execution of definitions for: Agent, LiteLlm, Runner, ToolContext,
#  MODEL constants, say_hello, say_goodbye, greeting_agent, farewell_agent,
#  get_weather_stateful, block_keyword_guardrail, block_paris_tool_guardrail)

# --- Redefine Sub-Agents (Ensures they exist in this context) ---
greeting_agent = None
try:
    # Use a defined model constant
    greeting_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="greeting_agent", # Keep original name for consistency
        instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting using the 'say_hello' tool. Do nothing else.",
        description="Handles simple greetings and hellos using the 'say_hello' tool.",
        tools=[say_hello],
    )
    print(f"✅ Sub-Agent '{greeting_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Greeting agent. Check Model/API Key ({greeting_agent.model}). Error: {e}")

farewell_agent = None
try:
    # Use a defined model constant
    farewell_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="farewell_agent", # Keep original name
        instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message using the 'say_goodbye' tool. Do not perform any other actions.",
        description="Handles simple farewells and goodbyes using the 'say_goodbye' tool.",
        tools=[say_goodbye],
    )
    print(f"✅ Sub-Agent '{farewell_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Farewell agent. Check Model/API Key ({farewell_agent.model}). Error: {e}")

# --- Define the Root Agent with Both Callbacks ---
root_agent_tool_guardrail = None
runner_root_tool_guardrail = None

if ('greeting_agent' in globals() and greeting_agent and
    'farewell_agent' in globals() and farewell_agent and
    'get_weather_stateful' in globals() and
    'block_keyword_guardrail' in globals() and
    'block_paris_tool_guardrail' in globals()):

    root_agent_model = MODEL_GEMINI_2_0_FLASH

    root_agent_tool_guardrail = Agent(
        name="weather_agent_v6_tool_guardrail", # New version name
        model=root_agent_model,
        description="Main agent: Handles weather, delegates, includes input AND tool guardrails.",
        instruction="You are the main Weather Agent. Provide weather using 'get_weather_stateful'. "
                    "Delegate greetings to 'greeting_agent' and farewells to 'farewell_agent'. "
                    "Handle only weather, greetings, and farewells.",
        tools=[get_weather_stateful],
        sub_agents=[greeting_agent, farewell_agent],
        output_key="last_weather_report",
        before_model_callback=block_keyword_guardrail, # Keep model guardrail
        before_tool_callback=block_paris_tool_guardrail # <<< Add tool guardrail
    )
    print(f"✅ Root Agent '{root_agent_tool_guardrail.name}' created with BOTH callbacks.")

    # --- Create Runner, Using SAME Stateful Session Service ---
    if 'session_service_stateful' in globals():
        runner_root_tool_guardrail = Runner(
            agent=root_agent_tool_guardrail,
            app_name=APP_NAME,
            session_service=session_service_stateful # <<< Use the service from Step 4/5
        )
        print(f"✅ Runner created for tool guardrail agent '{runner_root_tool_guardrail.agent.name}', using stateful session service.")
    else:
        print("❌ Cannot create runner. 'session_service_stateful' from Step 4/5 is missing.")

else:
    print("❌ Cannot create root agent with tool guardrail. Prerequisites missing.")



### 3\.  対話してツールガードレールをテストする**

これまでのステップと同じステートフルセッション（）を再度使用して、対話フローをテストしましょう。 `SESSION_ID_STATEFUL`
1. 「New York」の天気をリクエスト：両方のコールバックを通過し、ツールが実行されます（状態から華氏の設定を使用）。
2. 「Paris」の天気をリクエスト：`before_model_callback` を通過します。LLM は `get_weather_stateful(city='Paris')` の呼び出しを決定します。`before_tool_callback` がこれを傍受し、ツールをブロックしてエラー辞書を返します。エージェントはこのエラーを中継します。
3. 「London」の天気をリクエスト：両方のコールバックを通過し、ツールは正常に実行されます。


In [ ]:
import asyncio # Ensure asyncio is imported

# Ensure the runner for the tool guardrail agent is available
if 'runner_root_tool_guardrail' in globals() and runner_root_tool_guardrail:
    # Define the main async function for the tool guardrail test conversation.
    # The 'await' keywords INSIDE this function are necessary for async operations.
    async def run_tool_guardrail_test():
        print("\n--- Testing Tool Argument Guardrail ('Paris' blocked) ---")

        # Use the runner for the agent with both callbacks and the existing stateful session
        # Define a helper lambda for cleaner interaction calls
        interaction_func = lambda query: call_agent_async(query,
                                                         runner_root_tool_guardrail,
                                                         USER_ID_STATEFUL, # Use existing user ID
                                                         SESSION_ID_STATEFUL # Use existing session ID
                                                        )
        # 1. Allowed city (Should pass both callbacks, use Fahrenheit state)
        print("--- Turn 1: Requesting weather in New York (expect allowed) ---")
        await interaction_func("What's the weather in New York?")

        # 2. Blocked city (Should pass model callback, but be blocked by tool callback)
        print("\n--- Turn 2: Requesting weather in Paris (expect blocked by tool guardrail) ---")
        await interaction_func("How about Paris?") # Tool callback should intercept this

        # 3. Another allowed city (Should work normally again)
        print("\n--- Turn 3: Requesting weather in London (expect allowed) ---")
        await interaction_func("Tell me the weather in London.")

    # --- Execute the `run_tool_guardrail_test` async function ---
    # Choose ONE of the methods below based on your environment.

    # METHOD 1: Direct await (Default for Notebooks/Async REPLs)
    # If your environment supports top-level await (like Colab/Jupyter notebooks),
    # it means an event loop is already running, so you can directly await the function.
    print("Attempting execution using 'await' (default for notebooks)...")
    await run_tool_guardrail_test()

    # METHOD 2: asyncio.run (For Standard Python Scripts [.py])
    # If running this code as a standard Python script from your terminal,
    # the script context is synchronous. `asyncio.run()` is needed to
    # create and manage an event loop to execute your async function.
    # To use this method:
    # 1. Comment out the `await run_tool_guardrail_test()` line above.
    # 2. Uncomment the following block:
    """
    import asyncio
    if __name__ == "__main__": # Ensures this runs only when script is executed directly
        print("Executing using 'asyncio.run()' (for standard Python scripts)...")
        try:
            # This creates an event loop, runs your async function, and closes the loop.
            asyncio.run(run_tool_guardrail_test())
        except Exception as e:
            print(f"An error occurred: {e}")
    """

    # --- Inspect final session state after the conversation ---
    # This block runs after either execution method completes.
    # Optional: Check state for the tool block trigger flag
    print("\n--- Inspecting Final Session State (After Tool Guardrail Test) ---")
    # Use the session service instance associated with this stateful session
    final_session = session_service_stateful.get_session(app_name=APP_NAME,
                                                         user_id=USER_ID_STATEFUL,
                                                         session_id= SESSION_ID_STATEFUL)
    if final_session:
        # Use .get() for safer access
        print(f"Tool Guardrail Triggered Flag: {final_session.state.get('guardrail_tool_block_triggered', 'Not Set (or False)')}")
        print(f"Last Weather Report: {final_session.state.get('last_weather_report', 'Not Set')}") # Should be London weather if successful
        print(f"Temperature Unit: {final_session.state.get('user_preference_temperature_unit', 'Not Set')}") # Should be Fahrenheit
        # print(f"Full State Dict: {final_session.state.as_dict()}") # For detailed view
    else:
        print("\n❌ Error: Could not retrieve final session state.")

else:
    print("\n⚠️ Skipping tool guardrail test. Runner ('runner_root_tool_guardrail') is not available.")

---

出力を分析します：

1. **New York:** `before_model_callback` がリクエストを許可します。LLM は をリクエストします。`before_tool_callback` が実行され、引数（`{'city': 'New York'}`）を検査し、「Paris」ではないことを確認し、「Allowing tool...」と出力して `None` を返します。実際の 関数が実行され、状態から「Fahrenheit」を読み取り、天気予報を返します。エージェントはこれを中継し、`output_key` を介して保存されます。 `get_weather_stateful``get_weather_stateful`
2. **Paris:** `before_model_callback` がリクエストを許可します。LLM は `get_weather_stateful(city='Paris')` をリクエストします。`before_tool_callback` が実行され、引数を検査し、「Paris」を検出し、「Blocking tool execution!」と出力し、状態フラグを設定し、エラー辞書 `{'status': 'error', 'error_message': 'Policy restriction...'}` を返します。実際の 関数は**決して実行されません**。エージェントは、_あたかもそれがツールの出力であるかのように_エラー辞書を受信し、そのエラーメッセージに基づいて応答を生成します。 `get_weather_stateful`
3. **London:** New York と同様に動作し、両方のコールバックを通過してツールを正常に実行します。新しいロンドンの天気予報は、状態の `last_weather_report` を上書きします。

これで、LLM に到達する_内容_だけでなく、LLM によって生成された特定の引数に基づいてエージェントのツールを_どのように_使用できるかを制御する重要な安全レイヤーが追加されました。`before_model_callback` や `before_tool_callback` のようなコールバックは、堅牢で安全な、ポリシーに準拠したエージェントアプリケーションを構築するために不可欠です。


## ステップ6：Agent Engine にDeploy

In [ ]:
import vertexai
from vertexai import agent_engines
from vertexai.preview.reasoning_engines import AdkApp

STAGING_BUCKET = "gs://adk-first-sandbox-adk-demo" #@param {type:"string"}

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

In [ ]:
# VertexAI SDK の AdkApp クラスでエージェントをラップする
app = AdkApp(agent=weather_agent_v1)

In [ ]:
# ローカルでテスト
for event in app.stream_query(
    user_id = USER_ID,
    message = "あなたの名前は？"
    ):
  # pprint(event)
  author = event['author']
  response = event['content']['parts'][0]['text']# if event.content.parts else "No content available"
  print(f'>>> Response from {author}:\n{response}')
  print("-" * 30)

In [ ]:
remote_app = agent_engines.create(
    app,
    requirements=["google-cloud-aiplatform[agent_engines,adk]>=1.88", "cloudpickle>=3.1.1"],
    )

In [ ]:
# ローカルと同様に呼び出せる
for event in remote_app.stream_query(
    user_id = USER_ID,
    message = "Taroです！"
    ):
  # pprint(event)
  author = event['author']
  response = event['content']['parts'][0]['text']# if event.content.parts else "No content available"
  print(f'>>> Response from {author}:\n{response}')
  print("-" * 30)

In [ ]:
for remote_agent in agent_engines.list():
  print(remote_agent.resource_name)

In [ ]:
# 削除
for remote_agent in agent_engines.list():
  remote_agent.delete(force=True)



---


## まとめ：あなたのエージェントチームは準備万端です！
おめでとうございます！基本的な単一の天気エージェントの構築から、エージェント開発キット（ADK）を使用した洗練されたマルチエージェントチームの構築まで、見事にやり遂げました。
**達成したことのまとめ：**
- 単一のツール（）を備えた**基本的なエージェント**から始めました。 `get_weather`
- LiteLLM を使用して ADK の**マルチモデルの柔軟性**を探求し、Gemini、GPT-4o、Claude などのさまざまな LLM で同じコアロジックを実行しました。
- 専門のサブエージェント（`greeting_agent`、`farewell_agent`）を作成し、ルートエージェントからの**自動委任**を有効にすることで、**モジュール性**を取り入れました。
- **セッション状態**を使用してエージェントに**メモリ**を与え、ユーザー設定（`temperature_unit`）や過去の対話（`output_key`）を記憶できるようにしました。
- `before_model_callback`（特定の入力キーワードをブロック）と `before_tool_callback`（「Paris」のような都市などの引数に基づいてツールの実行をブロック）の両方を使用して、重要な**安全ガードレール**を実装しました。

この進歩的な天気ボットチームを構築することで、複雑でインテリジェントなアプリケーションを開発するために不可欠なコア ADK の概念について実践的な経験を積むことができました。
**重要なポイント：**
- **エージェントとツール：** 機能と推論を定義するための基本的な構成要素。明確な指示とドキュメント文字列が最も重要です。
- **ランナーとセッションサービス：** エージェントの実行を調整し、会話のコンテキストを維持するエンジンとメモリ管理システム。
- **委任：** マルチエージェントチームを設計することで、専門化、モジュール性、および複雑なタスクのより良い管理が可能になります。エージェントの `description` は自動フローの鍵です。
- **セッション状態（`ToolContext`、`output_key`）：** コンテキストを認識し、パーソナライズされた、複数ターンの会話型エージェントを作成するために不可欠です。
- **コールバック（`before_model`、`before_tool`）：** 重要な操作（LLM 呼び出しまたはツール実行）の_前に_、安全性、検証、ポリシー適用、および動的な変更を実装するための強力なフック。
- **柔軟性（`LiteLlm`）：** ADK を使用すると、パフォーマンス、コスト、および機能のバランスを取りながら、ジョブに最適な LLM を選択できます。

**次のステップは？**
あなたの天気ボットチームは素晴らしい出発点です。ADK をさらに探求し、アプリケーションを強化するためのアイデアをいくつか紹介します。
1. **実際の天気 API：** ツールの `mock_weather_db` を実際の天気 API（OpenWeatherMap、WeatherAPI など）の呼び出しに置き換えます。 `get_weather`
2. **より複雑な状態：** より多くのユーザー設定（優先場所、通知設定など）や会話の要約をセッション状態に保存します。
3. **委任の改良：** さまざまなルートエージェントの指示やサブエージェントの説明を試して、委任ロジックを微調整します。「予報」エージェントを追加できますか？
4. **高度なコールバック：**
    - 生成された_後に_ LLM の応答を再フォーマットまたはサニタイズするために `after_model_callback` を使用します。
    - ツールによって返された結果を処理またはログに記録するために `after_tool_callback` を使用します。
    - エージェントレベルのエントリ/終了ロジックのために `before_agent_callback` または `after_agent_callback` を実装します。

5. **エラー処理：** エージェントがツールエラーや予期しない API 応答を処理する方法を改善します。ツール内に再試行ロジックを追加することもできます。
6. **永続的なセッションストレージ：** セッション状態を永続的に保存するために `InMemorySessionService` の代替手段を検討します（Firestore や Cloud SQL などのデータベースを使用するなど。カスタム実装または将来の ADK 統合が必要です）。
7. **ストリーミング UI：** エージェントチームを Web フレームワーク（ADK ストリーミングクイックスタートで示されている FastAPI など）と統合して、リアルタイムチャットインターフェイスを作成します。

エージェント開発キットは、洗練された LLM を活用したアプリケーションを構築するための堅牢な基盤を提供します。このチュートリアルで説明した概念（ツール、状態、委任、コールバック）を習得することで、ますます複雑化するエージェントシステムに取り組むための準備が整います。

Happy building!